In [1]:
import numpy as np
from langchain_community.cross_encoders.huggingface import HuggingFaceCrossEncoder
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain.chat_models import ChatOpenAI

In [18]:
from chatsky_llm_autoconfig.graph import BaseGraph, Graph
from chatsky_llm_autoconfig.schemas import CompareResponse
from chatsky_llm_autoconfig.algorithms.three_stages_graph_generation import ThreeStagesGraphGenerator
from chatsky_llm_autoconfig.dialogue import Dialogue

In [3]:
from settings import EnvSettings
from utils import call_llm_api, read_json
from compare_prompt import (
    compare_graphs_prompt, graph_example_1, result_form
)

In [4]:
env_settings = EnvSettings()


In [5]:
evaluator = HuggingFaceCrossEncoder(model_name=env_settings.RERANKER_MODEL, model_kwargs={"device": env_settings.EMBEDDER_DEVICE})
graph_generator = ThreeStagesGraphGenerator()

In [6]:
def nodes2list(graph: dict) -> list:
    res = []

    for node in graph["nodes"]:
        utt = ""
        for n_utt in node['utterances']:
            utt += n_utt + " "
        res.append(utt)

    return res

In [7]:
def graph2list(graph: dict) -> tuple[list,int]:
    res = []
    n_edges = 0
    lens = []

    # node = [node for node in graph["nodes"] if node["is_start"]][0]
    for node in graph["nodes"]:
        edges = [e for e in graph['edges'] if e['source']==node["id"]]
        utt = ""
        for n_utt in node['utterances']:
            utt += n_utt + " "
        for edge in edges:
            lens.append(len(edge['utterances']))
            for e_utt in edge['utterances']:
                utt += e_utt + " "
                n_edges += 1
        res.append(utt)

        # node = [node for node in graph["nodes"] if node["id"]==edge['target']][0] 
    return res, n_edges, lens

In [8]:
def get_2_rerankings(generated1: list[str], golden1: list[str], generated2: list[str], golden2: list[str]):
    
    sz = len(generated1)
    to_score = []
    for gen in generated1:
        for gol in golden1:
            to_score.append((gen,gol))
    for gen in generated2:
        for gol in golden2:
            to_score.append((gen,gol))
    print("SCORING...")
    # print(to_score)
    scores = np.array(evaluator.score(to_score))
    print("finished")

    return scores[:sz*sz].reshape(sz,sz), scores[sz*sz:].reshape(sz,sz)

In [19]:
def llm_match(G1: BaseGraph, G2: BaseGraph) -> bool:
    g1 = G1.graph_dict
    g2 = G2.graph_dict

    # print("ORIG: ", g1)
    # g1_order = graph_order(g1)
    # g2_order = graph_order(g2)
    # print("ORDER: ", g1_order, "\n")
    # print("2LIST: ", graph2list(g1_order), "\n")
    #matrix = get_embedding(graph2list(g1_order), graph2list(g2_order), env_settings.EMBEDDER_MODEL, env_settings.EMBEDDER_DEVICE)

    nodes1_list = nodes2list(g1)
    nodes2_list = nodes2list(g2)
    if len(nodes1_list) != len(nodes2_list):
        return False

    g1_list, n1, len1 = graph2list(g1)
    g2_list, n2, len2 = graph2list(g2)

    # print("G1: ", g1_list, "\n")
    # print("G2: ", g2_list, "\n")

    nodes_matrix, matrix = get_2_rerankings(nodes1_list, nodes2_list, g1_list, g2_list)
    nodes_max = list(np.argmax(nodes_matrix, axis=1))
    if len(set(nodes_max)) < len(nodes1_list):
        return False


    # print("LENS: ", len1, len2)
    if n1 != n2:
        return False
    

    # matrix = get_reranking(g1_list, g2_list)
    max = list(np.argmax(matrix, axis=1))
    # print("MAX: ", max)
    if len(set(max)) < len(g1_list) or nodes_max != max:
        return False
    if any([len1[i] != len2[max[i]] for i in range(len(max))]):
        return False
    print("NODES: ", np.min(np.max(nodes_matrix, axis=1)))
    print("ALL: ", np.min(np.max(matrix, axis=1)))

    if min(np.min(np.max(nodes_matrix, axis=1)),np.min(np.max(matrix, axis=1))) >= env_settings.SIM_THRESHOLD:
        return True
    # diags = get_diagonals(matrix)
    # # print("DIAGS: ", diags, "\n")
    # sums = np.sum(diags,axis=1)
    # max_index = np.argmax(sums)
    # g1_best = get_diagonal(g1,max_index)
    # min_value = np.min(diags[max_index])
    # print("MIN: ", min_value)
    # return True
    # print("\nG1: ", g1_best, "\n")
    # print("G2: ", g2_order, "\n")

    # if min_value >= env_settings.SIM_THRESHOLD:
    #     return True
    parser = PydanticOutputParser(pydantic_object=CompareResponse)
    format_model=ChatOpenAI(model=env_settings.FORMATTER_MODEL_NAME, api_key=env_settings.OPENAI_API_KEY, base_url=env_settings.OPENAI_BASE_URL)
    model=ChatOpenAI(model=env_settings.COMPARE_MODEL_NAME, api_key=env_settings.OPENAI_API_KEY, base_url=env_settings.OPENAI_BASE_URL)
    new_parser = OutputFixingParser.from_llm(parser=parser, llm=format_model)
    result = call_llm_api(compare_graphs_prompt.format(result_form=result_form,graph_example_1=graph_example_1, graph_1=g1, graph_2=g2), model|new_parser, temp=0).model_dump()
    # print("RES: ", result)
    return result['result']



In [10]:
dialogue_to_graph = read_json(env_settings.TEST_DATA_PATH)
case = dialogue_to_graph[1]

In [11]:
result_graph = graph_generator.invoke([Dialogue.from_list(c["messages"]) for c in case["dialogues"]])

LISTS_N:  [(0, ['The size I ordered is not correct.']), (1, ["Sure, it's 123456.", "Never mind, I don't need assistance."]), (2, ['I would like a replacement.', "I changed my mind, I don't need any help.", 'I would like a refund.', 'Can I exchange it for another size?', 'I would like to exchange for a different size.', 'Actually, I need a different size.']), (3, ['Actually, can I change my choice?', "No, that's all, thank you."]), (4, ['']), (5, ['Actually, can I change my mind?', "No, that's all, thanks."]), (6, ['Actually, I want to choose another option instead.', "No, that's all, thank you."]), (7, ['Actually, I want to choose another option instead.', "No, that's all, thank you."]), (8, ['Actually, I want to choose another option instead.', "No, that's all, thank you."])]
LISTS:  [(0, "I'm sorry to hear that the size of the product you received isn't what you expected. How can I assist you today?"), (1, "I'm sorry for the inconvenience. Could you please provide your order number?"

/data/home/peshkichev/projects/chatsky-llm-autoconfig/dev_packages/chatsky_llm_autoconfig/chatsky_llm_autoconfig/algorithms/three_stages_graph_generation.py:54: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=env_settings.EMBEDDER_MODEL, model_kwargs={"device": env_settings.EMBEDDER_DEVICE})


TEXTS:  [{'text': "I'm sorry to hear that the size of the product you received isn't what you expected. How can I assist you today?", 'participant': 'assistant'}, {'text': 'The size I ordered is not correct.', 'participant': 'user'}, {'text': "I'm sorry for the inconvenience. Could you please provide your order number?", 'participant': 'assistant'}, {'text': "Sure, it's 123456.", 'participant': 'user'}, {'text': 'Thank you. I see that you ordered a Medium size. Would you like a replacement, a refund, or to exchange it for a different size?', 'participant': 'assistant'}, {'text': 'I would like a replacement.', 'participant': 'user'}, {'text': 'Understood. We will send you a replacement in size Medium. You should receive it within 5 business days. Is there anything else I can assist you with?', 'participant': 'assistant'}, {'text': 'Actually, can I change my choice?', 'participant': 'user'}, {'text': 'Thank you. I see that you ordered a Medium size. Would you like a replacement, a refund

No relevant docs were retrieved using the relevance score threshold 0.98
No relevant docs were retrieved using the relevance score threshold 0.98


find_node:  target:  3 Thank you. I see that you ordered a Medium size. Would you like a replacement, a refund, or to exchange it for a different size? 2 1 Sure, it's 123456.
ADDED:  {'source': 2, 'target': 3, 'utterances': ["Sure, it's 123456."]}
NODE:  {'id': 3, 'label': '', 'is_start': False, 'utterances': ['Thank you. I see that you ordered a Medium size. Would you like a replacement, a refund, or to exchange it for a different size?']}
UTT:  Thank you. I see that you ordered a Medium size. Would you like a replacement, a refund, or to exchange it for a different size?
IDS:  ['2', '4']
USER:  I would like a replacement.
find_node:  target:  4 Understood. We will send you a replacement in size Medium. You should receive it within 5 business days. Is there anything else I can assist you with? 3 2 I would like a replacement.
ADDED:  {'source': 3, 'target': 4, 'utterances': ['I would like a replacement.']}
USER:  I changed my mind, I don't need any help.
find_node:  target:  5 You're w

...

find_node:  target:  8 Sure, we can exchange your product for another size. Which size would you like instead? 3 2 Can I exchange it for another size?
NOOO
NODE:  {'id': 4, 'label': '', 'is_start': False, 'utterances': ['Understood. We will send you a replacement in size Medium. You should receive it within 5 business days. Is there anything else I can assist you with?']}
UTT:  Understood. We will send you a replacement in size Medium. You should receive it within 5 business days. Is there anything else I can assist you with?
IDS:  []
NODE:  {'id': 5, 'label': '', 'is_start': False, 'utterances': ["You're welcome! Have a great day."]}
UTT:  You're welcome! Have a great day.
IDS:  ['4']
NODE:  {'id': 6, 'label': '', 'is_start': True, 'utterances': ['Your refund has been processed. You should see the amount credited to your account within 3-5 business days. Is there anything else I can assist you with?']}
UTT:  Your refund has been processed. You should see the amount credited to your ac

No relevant docs were retrieved using the relevance score threshold 0.98
No relevant docs were retrieved using the relevance score threshold 0.98


ASSISTANT_STORE:  {'ids': ['0', '1', '2'], 'embeddings': None, 'documents': ["I'm sorry to hear that the size of the product you received isn't what you expected. How can I assist you today?", "I'm sorry for the inconvenience. Could you please provide your order number?", "You're welcome! Have a great day."], 'uris': None, 'data': None, 'metadatas': [{'id': 0}, {'id': 1}, {'id': 2}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
USER_STORE:  {'ids': ['0', '1'], 'embeddings': None, 'documents': ['The size I ordered is not correct.', "Never mind, I don't need assistance."], 'uris': None, 'data': None, 'metadatas': [{'id': 0}, {'id': 1}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
NODE:  {'id': 1, 'label': '', 'is_start': True, 'utterances': ["I'm sorry to hear that the size of the product you received isn't what you expected. How can I assist you today?"]}
UTT:  I'm sorry to hear that the size of t

No relevant docs were retrieved using the relevance score threshold 0.98
No relevant docs were retrieved using the relevance score threshold 0.98
No relevant docs were retrieved using the relevance score threshold 0.98
No relevant docs were retrieved using the relevance score threshold 0.98


IDS:  ['2']
NODE:  {'id': 6, 'label': '', 'is_start': True, 'utterances': ['Your refund has been processed. You should see the amount credited to your account within 3-5 business days. Is there anything else I can assist you with?']}
UTT:  Your refund has been processed. You should see the amount credited to your account within 3-5 business days. Is there anything else I can assist you with?
IDS:  []
NODE:  {'id': 7, 'label': '', 'is_start': True, 'utterances': ['Certainly, we can exchange your product for a different size. Please let us know which size you would prefer.']}
UTT:  Certainly, we can exchange your product for a different size. Please let us know which size you would prefer.
IDS:  []
NODE:  {'id': 8, 'label': '', 'is_start': True, 'utterances': ['Sure, we can exchange your product for another size. Which size would you like instead?']}
UTT:  Sure, we can exchange your product for another size. Which size would you like instead?
IDS:  []
NODE:  {'id': 9, 'label': '', 'is_st

In [16]:
result_graph.graph_dict

{'nodes': [{'id': 1,
   'label': '',
   'is_start': True,
   'utterances': ["I'm sorry to hear that the size of the product you received isn't what you expected. How can I assist you today?"]},
  {'id': 2,
   'label': '',
   'is_start': False,
   'utterances': ["I'm sorry for the inconvenience. Could you please provide your order number?"]},
  {'id': 3,
   'label': '',
   'is_start': False,
   'utterances': ['Thank you. I see that you ordered a Medium size. Would you like a replacement, a refund, or to exchange it for a different size?']},
  {'id': 4,
   'label': '',
   'is_start': False,
   'utterances': ['Understood. We will send you a replacement in size Medium. You should receive it within 5 business days. Is there anything else I can assist you with?']},
  {'id': 5,
   'label': '',
   'is_start': False,
   'utterances': ["You're welcome! Have a great day."]},
  {'id': 6,
   'label': '',
   'is_start': True,
   'utterances': ['Your refund has been processed. You should see the amou

In [17]:
case['graph']

{'edges': [{'source': 1,
   'target': 2,
   'utterances': ['The size I ordered is not correct.']},
  {'source': 2, 'target': 3, 'utterances': ["Sure, it's 123456."]},
  {'source': 2,
   'target': 7,
   'utterances': ["Never mind, I don't need assistance."]},
  {'source': 3, 'target': 4, 'utterances': ['I would like a replacement.']},
  {'source': 3, 'target': 5, 'utterances': ['I would like a refund.']},
  {'source': 3,
   'target': 6,
   'utterances': ['I would like to exchange for a different size.',
    'Actually, I need a different size.',
    'Can I exchange it for another size?']},
  {'source': 3,
   'target': 7,
   'utterances': ["I changed my mind, I don't need any help."]},
  {'source': 4, 'target': 7, 'utterances': ["No, that's all, thank you."]},
  {'source': 4,
   'target': 3,
   'utterances': ['Actually, can I change my choice?']},
  {'source': 5, 'target': 7, 'utterances': ["No, that's all, thanks."]},
  {'source': 5,
   'target': 3,
   'utterances': ['Actually, can I cha

In [20]:
llm_match(Graph(case['graph']),result_graph)

SCORING...


finished


ValueError: cannot reshape array of size 77 into shape (7,7)